In [ ]:
from google.colab import drive

#Conect your gmail drive to the netbook
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
import os
import cv2
from datetime import datetime
from tifffile import imwrite
from collections import defaultdict


#Run the function on a folder containing several measurements.

In [ ]:
# Input and output paths
PATHinput = '/content/drive/MyDrive/NPEC_notebooks/Hyperspectral/Input_Examples/Example900to1700wl/'
PATHoutput = '/content/drive/MyDrive/NPEC_notebooks/Hyperspectral/Output/'

# Pattern that separates measurement name from data label (e.g. data, dark, white)
#Example my file name is 31_12_2025-03-12_23-32-36_G8_1x1_square_537_SWIR_WhiteCalibration.hdr
#before SWIR is my measurement data after that is the type of data

FILE_NAME_key = "SWIR" #Note FILE_Name_key should be around _

Data_Suffix ='Data.bil'
White_Suffix = 'WhiteCalibration.bil'
Dark_Suffix = 'DarkCalibration.bil'

In [ ]:
files = os.listdir(PATHinput)
groups = defaultdict(list)

for f in files:
    base, ext = os.path.splitext(f)
    parts = base.split('_')

    if FILE_NAME_key in parts:
        swir_index = parts.index(FILE_NAME_key)
        # Group key is everything before 'SWIR'
        group_key = '_'.join(parts[:swir_index])
        groups[group_key].append(f)

# === Parameters from your header info === These are usually all the same across PSI SWIR cameras
rows_data = 360
rows_cal = 5
cols = 510
bands = 318
start_wavelength = 901.465
end_wavelength = 1690.67

# Generate wavelengths vector
wavelengths = np.linspace(start_wavelength, end_wavelength, bands)
# Load BIL data function

def load_bil_data(PATHinput0, group_key0, FILE_NAME_key0, suffix0, rows, cols, bands):
    file_name0 = f'{PATHinput0}{group_key0}_{FILE_NAME_key0}_{suffix0}'
    data = np.fromfile(file_name0, dtype='<u2')  # little-endian uint16
    data = data.reshape((rows, bands, cols))
    return data

def find_band(target_wavelength):
    return np.argmin(np.abs(wavelengths - target_wavelength))


def save_tiff(img, filename):
    # tifffile.imwrite doesn't have dpi parameter, but we can embed metadata
    # Here we save raw array as 32-bit float (for precision)
    tifffile.imwrite(filename, img.astype('float32'))

def ImgParameters(PATHinput0, group_key0, FILE_NAME_key0, Data_Suffix0, Dark_Suffix0, White_Suffix0):
  # Load raw and calibration data
  raw = load_bil_data(PATHinput0, group_key0, FILE_NAME_key0, Data_Suffix0, rows_data, cols, bands)
  dark = load_bil_data(PATHinput0, group_key0, FILE_NAME_key0, Dark_Suffix0, rows_cal, cols, bands)
  white = load_bil_data(PATHinput0, group_key0,FILE_NAME_key0, White_Suffix0, rows_cal, cols, bands)
  #white
  white_avg_spectrum = white.mean(axis=(0, 2))
  white_cube_expanded = np.tile(white_avg_spectrum[np.newaxis, :, np.newaxis], (rows_data, 1, cols))
  #dark
  dark_avg_spectrum = dark.mean(axis=(0, 2))
  dark_cube_expanded = np.tile(dark_avg_spectrum[np.newaxis, :, np.newaxis], (rows_data, 1, cols))
  #calibration
  epsilon = 1e-8
  calibrated = (raw - dark_cube_expanded) / (white_cube_expanded - dark_cube_expanded + epsilon)
  calibrated = np.transpose(calibrated, (0, 2, 1))
  calibrated = np.clip(calibrated, 0, 1)
  #bands of interst
  nir_band = find_band(900)          # NIR (within 860–950 nm range)
  swir1_band = find_band(1250)       # SWIR1 (within 1230–1300 nm range)
  swir2_band = find_band(1650)       # SWIR2 (within 1600–1700 nm range)
  wl1000_band = find_band(1000)
  #filter data
  nir = calibrated[:, :, nir_band]   # shape: (360, 510)
  swir1 = calibrated[:, :, swir1_band]
  swir2 = calibrated[:, :, swir2_band]
  wl1000 = calibrated[:, :, wl1000_band]
  #Parameters images
  #NDWI: Normalized Difference Water Index
  ndwi = (nir - swir1) / (nir + swir1 + 1e-6)
  # MSI: Moisture Stress Index
  msi = swir1 / (nir + 1e-6)
  # NDII: Normalized Difference Infrared Index
  ndii = (nir - swir2) / (nir + swir2 + 1e-6)
  # Save each index
  save_tiff(ndwi, f'{PATHoutput}{group_key}ndwi.tiff')
  save_tiff(msi, f'{PATHoutput}{group_key}msi.tiff')
  save_tiff(ndii, f'{PATHoutput}{group_key}ndii.tiff')
  save_tiff(wl1000, f'{PATHoutput}{group_key}wl1000.tiff')
  print(f"Processed and saved indices for group: {group_key0}")

In [ ]:
# Print groups info
for group_key, group_files in groups.items():
  print(group_key)
  ImgParameters(PATHinput, group_key, FILE_NAME_key, Data_Suffix, Dark_Suffix, White_Suffix)


31_12_2025-03-12_23-32-36_G8_1x1_square_537
Processed and saved indices for group: 31_12_2025-03-12_23-32-36_G8_1x1_square_537
31_12_2025-03-12_20-15-47_G8_1x1_square_302
Processed and saved indices for group: 31_12_2025-03-12_20-15-47_G8_1x1_square_302
